# Investigating the role of pre-miRNA concentration on competition

Using same model as for the investigations into the competition effects between 8 theoretical pre-miRNAs

## Set up workspace

In [1]:
import model_utils as mu
import params
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from tqdm import tqdm
import seaborn as sns
import matplotlib.style as style
style.use('seaborn-colorblind')

In [2]:
destdir_csvs = './outputs/premir-conc-vars/csvs/'
destdir_figs = './outputs/premir-conc-vars/plots/'

## Generate model

In [3]:
#make 2 miRNA model
ODE_inits = [0]*8*4 + [0]
ODE_ka = [0] * 8
ODE_kb = ODE_ka
ODE_kc = ODE_ka
ODE_kd = ODE_ka

In [4]:
ode = mu.makeODEs(ODE_inits, ODE_ka, ODE_kb, ODE_kc, ODE_kd)

In [5]:
model = mu.makeModel(ode)

## Set up conditions for models

In [6]:
high = 10
low = 1
k = 2

In [7]:
inits1 = [params.init_pMiR1, params.init_pMiR1_dcr, params.init_pMiR1_dcr_star, params.init_MiR1] * 8

In [8]:
ka = [params.ka1 * high, params.ka1 * low] + [params.ka1] * 2 +\
     [params.ka1 * high * k, params.ka1 * low] + [params.ka1 * k] * 2

kb = [params.ka1 * params.Kd_wt, params.ka1 * params.Kd_wt * high]  + \
    [params.ka1 * params.Kd_wt] * 2 + \
    [params.ka1 * params.Kd_wt, params.ka1 * params.Kd_wt * high * k] +\
    [params.ka1 * params.Kd_wt] * 2

kc = [params.kc1] * 2 + [params.kc1 * high, params.kc1] + \
     [params.kc1] * 2 + [params.kc1 * high * k, params.kc1]

kd = [params.kd] * 3 + [params.kd * high] + [params.kd] * 3 + [params.kd * high * k]

In [9]:
#Dicer concentration variations
dvars = np.linspace(0, 7.25, 15)
dvars[0] = 0.01
dvars = np.append(dvars, np.linspace(10, 15, 4))
variations = [str(int(i * 100)) for i in list(dvars)]

In [17]:
#pre-miRNA concentration variations
pvars = np.linspace(0, 1, 5)
pvars = np.append(pvars, np.linspace(1.5, 5, 8))
p_variations = [str(int(i*100)) for i in list(dvars)]

## Run model, save outputs

In [19]:
exec(model)

In [22]:
names = []
for ii in tqdm(range(len(dvars))):
    for ij in range(8):
        for ik in range(len(pvars)):
            name = 'model_' + variations[ii] + '_miR' + str(ij) + '_' + p_variations[ik]
            
            init = inits1 + [dvars[ii]]
            init[ij] = init[ij] * pvars[ik]
            
            res = solve_ivp(runModel, (0, int(params.minutes)), init, \
                            args = (ka, kb, kc, kd), method = 'LSODA')
    
            mirna_length = int((len(res.y)-1)/4)
            index = []
    
            for j in range(mirna_length):
                index = index + ['pMiR' + str(j), 'pMiR_dicer' + str(j), \
                                 'pMiR_dicer_star' + str(j), 'miR' + str(j)]
    
            index = index + ['dicer']
    
            df = pd.DataFrame(res.y, index = index)
            df = df.transpose()
            df.insert(loc = 0, column = 'time_min', value = res.t)
    
            names.append(name)
    
            df.to_csv(destdir_csvs + name + '.csv', index = False)

df1 = pd.DataFrame({'names': names})
df1.to_csv(destdir_csvs + 'names.csv', index = False)

100%|██████████| 19/19 [00:58<00:00,  3.09s/it]
